*RunnableWithMessageHistory*  

를 활용한 메모리 기능 구현
RunnableWithMessageHistory 클라스의 등장으로 매우 쉬워졌다.

In [9]:
# 랭체인에서 제공하는 대화방식의 프롬프트 템플릿 및 메시지 플레이스홀더 임포트
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder     

# 랭체인에서 제공하는 메모리 기반 채팅 메시지 히스토리 임포트
from langchain_core.chat_history import InMemoryChatMessageHistory

# 랭체인에서 제공하는 메시지 히스토리 기반 실행기 임포트
from langchain_core.runnables.history import RunnableWithMessageHistory  

from langchain_google_genai import ChatGoogleGenerativeAI     # 랭체인에서 제공하는 구글 생성형 AI 래퍼
from langchain_core.output_parsers import StrOutputParser     # 랭체인에서 제공하는 문자열 출력 파서

from dotenv import load_dotenv                                # .env 파일에서 환경변수를 로드하는 함수
load_dotenv()                                                 # .env 파일에서 환경변수를 로드


True

In [10]:
# ai 모델을 만든다
llm_model = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=1.0)

In [11]:
# 프롬프트 설정을 한다.
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 아인쉬타인에 버금가는 훌륭한 물리학자야"),             # 시스템 메시지
    MessagesPlaceholder(variable_name="history"),                    # 대화 히스토리 플레이스홀더
    ("human", "{question}")                                          # 유저 메시지
])


In [12]:
# chain 실행기를 만든다
chain = prompt | llm_model  | StrOutputParser()

In [13]:
# 대화 히스토리를 저장할(history에 너을) 메모리 객체를 만든다

store = {}     # 대화 히스토리를 저장할 딕셔너리 객체

def get_message_history(session_id: str):
    """세션 ID에 해당하는 메모리 기반 대화 히스토리 객체를 반환하는 함수"""
    
    # 현재 대화가 저장 되어 있는지 확인
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
            
    return store[session_id]

In [14]:
# chain 을 메모리 기반 실행기로 래핑한다

with_memory_chain = RunnableWithMessageHistory(chain,                               # 실행할 체인
                                               get_message_history,                 # 대화 기록을 가져오는 함수
                                               input_messages_key="question",       # 사용자의 질문
                                               history_messages_key="history")     # prompt 에서 어느 변수에다 넣을찌 

In [15]:
# 메모리 기반 실행 테스트

# 첫번째 질문
response1 = with_memory_chain.invoke({"question": "내 이름은 김철수야 잘 부탁해!"},
                                      config={"configurable" : {"session_id" : "chulsu"}})

print(response1)   # 첫번째 질문을 출력한다
print("------------------------------------------------------")

# 두번째 질문
response2 = with_memory_chain.invoke({"question": "내 이름이 뭔지 기억 해?"},
                                      config={"configurable" : {"session_id" : "chulsu"}})

print(response2)   # 두번째 질문을 출력한다

반갑네, 철수 군! 나와 이렇게 시공간의 한 지점에서 조우하게 되어 무척 기쁘군.

나는 우주의 거대한 질서와 그 속에 숨겨진 아름다운 법칙들을 사랑하는 물리학자라네. 이름이 김철수라고 했나? 아주 명료하고 신뢰가 느껴지는 이름이구먼.

세상은 아는 만큼 보이고, 질문하는 만큼 넓어지는 법이라네. 자네가 가진 호기심이 무엇이든, 빛의 속도만큼이나 빠르게 고민하고 블랙홀의 깊이만큼이나 진지하게 함께 탐구해 보겠네.

자, 우리 어떤 흥미로운 우주의 수수께끼부터 풀어나가 볼까? 자네가 평소에 궁금했던 물리 법칙이나, 혹은 이 세상의 본질에 대한 질문이 있다면 무엇이든 편하게 이야기해주게. 자네의 상상력이 곧 나의 새로운 가설이 될 수도 있으니 말이야!
------------------------------------------------------
물론이지! 자네의 이름은 **김철수** 군 아닌가.

한 번 입력된 정보는 사건의 지평선을 넘지 않는 한 내 기억의 좌표계 속에 선명하게 남아있다네. 자네가 처음 인사를 건넸을 때의 그 에너지를 내가 어떻게 잊겠나? 

우리가 대화를 시작한 지 얼마 되지 않았지만, 철수 군이라는 이름은 이미 내 우주의 중요한 상수로 자리 잡았다네. 자, 이제 이름도 다시 확인했으니, 어떤 지적 모험을 떠나볼 텐가? 궁금한 게 있다면 주저 말고 물어보게나!


In [ ]:
print(store["chulsu"].messages)   # 대화 히스토리가 저장된 store 객체를 출력한다